# Load in the data

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

raw_data = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical00.txt', sep='|')

raw_data.head()


Mounted at /content/drive


,ID,VERSION,V00BLDCOLL,V00BLDHRS1,V00BLDHRS2,V00BLDRAW1,V00BLDRAW2,V00BLSURD1,V00BLSURD2,V00CITRATE,V00EDTA,V00excess1,V00excess2,V00hemat1,V00hemat2,V00hoursp1,V00hoursp2,V00hrsuc1,V00hrsuc2,V00illpwk1,V00illpwk2,V00LEAKAG1,V00LEAKAG2,V00MRSEQNL,V00MRSEQNR,V00MULTST1,V00MULTST2,V00othvp1,V00othvp2,V00pdate1,V00pdate2,V00PLAQHR1,V00PLAQHR2,V00qovp1,V00qovp2,V00SEAQHR1,V00SEAQHR2,V00SERUM,V00ucdate1,V00ucdate2,...,V00HOURWK,V00INCOME,V00INCOME2,V00LIVE1,V00LIVE2,V00LIVE3,V00LIVE4,V00LIVE5,V00LIVENO,V00MARITST,V00MEDINS,V00MISSWK,V00PA130,V00PA130CV,V00PA130NM,V00PA230,V00PA230CV,V00PA330,V00PA330CV,V00PA430,V00PA430CV,V00PA530,V00PA530CV,V00PASE,V00PASE1,V00PASE1HR,V00PASE2,V00PASE2HR,V00PASE3,V00PASE3HR,V00PASE4,V00PASE4HR,V00PASE5,V00PASE5HR,V00PASE6,V00PASE6HR,V00WEEKWK,V00WKHR7CV,V00WORK7,V00WORKAMT
0,9000099,0.2.3,1: Sample at 1st collection only,35400.0,NaN,1: Yes,.: Missing Form/Incomplete Workbook,NaN,NaN,1: Sample at 1st collection only,1: Sample at 1st collection only,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,11.0,NaN,8.3,NaN,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,5: 5,6: 6,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,16625.0,NaN,36900.0,NaN,1: Clean,.: Missing Form/Incomplete Workbook,38400.0,NaN,1: Sample at 1st collection only,16625.0,NaN,...,50.0,4: $50K to < $100K,2: > $50K,1: Yes,.: Missing Form/Incomplete Workbook,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,2: Two,1: Married,1: Yes,0.0,0: No,0: None,5: More than 6 flights,0: No,0: None,0: No,0: None,1: Yes,1: One day per week or less,0: No,0: None,132.0,3: Often (5-7 days),2: 1 hour but less than 2 hours,3: Often (5-7 days),1: Less than 1 hour,0: Never,.: Missing Form/Incomplete Workbook,0: Never,.: Missing Form/Incomplete Workbook,3: Often (5-7 days),1: Less than 1 hour,2: Sometimes (3-4 days),1: Less than 1 hour,52.0,50.0,1: Yes,1: Sitting
1,9000296,0.2.3,1: Sample at 1st collection only,30780.0,NaN,1: Yes,.: Missing Form/Incomplete Workbook,NaN,NaN,1: Sample at 1st collection only,1: Sample at 1st collection only,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,10.0,NaN,8.0,NaN,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,5: 5,6: 6,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,16323.0,NaN,33000.0,NaN,1: Clean,.: Missing Form/Incomplete Workbook,34800.0,NaN,1: Sample at 1st collection only,16323.0,NaN,...,50.0,3: $25K to < $50K,1: < $50K,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,1: One,1: Married,1: Yes,0.0,0: No,0: None,5: More than 6 flights,0: No,0: None,0: No,0: None,1: Yes,3: 4-5 days per week,0: No,0: None,190.0,3: Often (5-7 days),2: 1 hour but less than 2 hours,2: Sometimes (3-4 days),2: 1 hour but less than 2 hours,0: Never,.: Missing Form/Incomplete Workbook,0: Never,.: Missing Form/Incomplete Workbook,0: Never,.: Missing Form/Incomplete Workbook,0: Never,.: Missing Form/Incomplete Workbook,52.0,50.0,1: Yes,2: Sitting/standing/walking
2,9000622,0.2.3,1: Sample at 1st collection only,33900.0,NaN,1: Yes,.: Missing Form/Incomplete Workbook,NaN,NaN,1: Sample at 1st collection only,1: Sample at 1st collection only,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,14.0,NaN,12.5,NaN,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,5: 5,6: 6,0: No,.: Missing Form/Incomplete Workbook,0: No,.: Missing Form/Incomplete Workbook,16625.0,NaN,35100.0,NaN,1: Clean,.: Missing Form/Incomplete Workbook,37200.0,NaN,1: Sample at 1st collection only,16625.0,NaN,...,NaN,2: $10K to < $25K,1: < $50K,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Inc

In [ ]:
raw_data.shape

(4796, 1187)

In [ ]:
# Initial dictionnary with following format: {name: index}
initial_dict = {}
for i in range(len(raw_data.columns)):
  initial_dict[raw_data.columns[i]] = i

# First features selection

In [ ]:
# Lists of kept features
# The lists correspond to each member of the group, because we splitted the features to analyzed among us

list_features_Aish = ['ID','P01KPNREV','P01KPNLEV','P01KPACT30','P01HPNR12','P01HPNL12','V00SF2','V00SF3','V00SF8','V00WPRKN1','V00WPRKN2','V00P7RKFR','V00KSXRKN1','V00DIRKN1','V00DIRKN2','V00DIRKN14','V00WPLKN1','V00WPLKN2','V00P7LKFR','V00KSXLKN1','V00DILKN1','V00DILKN2','V00DILKN14','V00KOOSFX5','P02KPNRCV','P02KPNLCV','P01KPR30CV','P01KPL30CV','P01KPACTCV','P01HPR12CV','P01HPL12CV','P01KPA30CV','V00P7RKRCV','V00P7LKRCV','P01PMRKRCV','P01PMLKRCV','V00WOMTSL','V00WOMTSR']
list_features_Antoine = ['P01BL12SXL', 'P01BL12SXR', 'P01KPACDCV', 'P01LKP30CV', 'P01RKP30CV', 'P02KPMED', 'P02KSURG', 'P01RAIA', 'P01RAJS1HR', 'P01RAKN6R', 'P01RAKN6L', 'P01ARTHOTH', 'P01ARTDOC', 'P01KPMED', 'P01INJR', 'P01KSURGR', 'P01KSURGL', 'V00HRTAT', 'V00STROKE', 'V00DIAB', 'V00RA', 'V00RAMEDS', 'V00POLYRH', 'V00BONEFX', 'V00SMOKE', 'V00CAM12', 'P02KPMEDCV', 'P01OADEGCV', 'P01OAHIPCV']
list_features_Oriane = ['P01OADEGCV','P01OAHIPCV','P01GOUTCV','P01OTARTCV','P01ARTDRCV','P01KPMEDCV','V00COMORB','V00FALLCV','V00HT25MM','V00WT25KG','V00WTMAXKG','V00WTMINKG','V00HYINJCV','V00STINJCV','V00ACUTCV','V00ACUSCV','V00CHIRCV','V00HOMECV','V00MASSCV','V00DIETCV','V00VITMCV','V00RUBCV','V00CAPSNCV','V00BRACCV','V00YOGACV','V00HERBCV','V00RELACV','V00SPIRCV','P01RASTASV','P02KSURGCV','P02CNCOTH','P01RAWE6L','P01HRSRFXO','P01HRSLFXO','P01RAWE6R','V00OTHCAMC','V00OTHCAM']
list_features_Vijitha = ['V00DISCOMF', 'V00REASW12', 'V00REASW10', 'V00REASW9', 'V00LFXCOMP', 'V00LFXPN', 'V00EKRSL', 'V00REXPN', 'V00EKRSR', 'V00RFXCOMP', 'V00BPDIAS', 'V00BPSYS', 'V00DTVITD', 'V00DTCALC', 'V00DTCHOL', 'V00VITDCV', 'V00CALCMCV', 'V00VIT9', 'V00VIT1', 'V00FFQYR86' ]
list_features_William = ['V00RPWKTYP','V00LPWKTYP','V00NPN400W', 'P01BMI','V00PASE','P01FAMKR','P02WTGA','V00PN400W','V00OTH400W','V00RKEFFB','V00RKEFFPT','V00LKEFFB','V00LKEFFPT','V00RKRFXPN','V00LKRFXPN','V00LLWGT','V00RLWGT']

list_features = list_features_Aish + list_features_Antoine + list_features_Oriane + list_features_Vijitha + list_features_William

In [ ]:
## Create a dictionnary with all kept features, to track original index and new index
## {original index: new index} for every feature kept

dictionary_track = {}

for i in range(len(list_features)):
  original_index = initial_dict[list_features[i]]
  new_index = i
  dictionary_track[new_index] = original_index

In [ ]:
# Creation of the new dataset with the selected features

selected_data = pd.DataFrame()

for i in list_features:
  selected_data[i] = raw_data[i]

selected_data.head()
  

,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,P01RAWE6R,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00REASW12,V00REASW10,V00REASW9,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,V00FFQYR86,V00RPWKTYP,V00LPWKTYP,V00NPN400W,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00PN400W,V00OTH400W,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT
0,9000099,1: Yes,1: Yes,1: Yes,0: No,0: No,3: Not limited at all,"2: Yes, limited a little",1: Not at all,0: None,1: Mild,4: Always,0: Never,1: Mild,1: Mild,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,1: Yes,0: No,1: Yes,1: Yes,2: Limits,0: No,0: No,1: Yes,4: 4,0: No pain,3: 3,2: 2,0.0,14.0,1: SV only,3: Both IEI and SV,...,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,1: Yes,0: No,0: No,1: Yes,0: No,1: Yes,60.0,124.0,276.00,498.9,258.9,0: No vitamins/minerals taken in past year,0: No vitamins/minerals taken in past year,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,23.8,132.0,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,0: No,1: Yes,0: No,22.0,23.0
1,9000296,1: Yes,0: No,0: No,1: Yes,0: No,3: Not limited at all,3: Not limited at all,1: Not at all,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,0: None,0: No,0: No,0: No,0: No,0: No Limits or avoidance,1: Yes,0: No,0: No,0: No pain,0: No pain,0: No pain,0: No pain,0.0,0.0,4: Neither,4: Neither,...,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,1: Yes,0: No,0: No,0: No,0: No,1: Yes,84.0,152.0,73.00,495.5,260.7,1: Didn t take,1: Didn t take,0: No,1: Yes,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,29.8,190.0,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,0: No,0: No,0: No,12.0,18.0
2,9000622,1: Yes,0: No,1: Yes,1: Yes,0: No,"2: Yes, limited a little","2: Yes, limited a little",3: Moderately,1: Mild,1: Mild,3: Daily,2: Sometimes,0: None,1: Mild,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,1: Yes,0: No,1: Yes,0: No,2: Limits,1: Yes,0: No,1: Yes,3: 3,0: No pain,2: 2,0: No pain,0.0,20.9,4: Neither,3: Both IEI and SV,...,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,1: Yes,0: No,0: No,0: No,0: No,1: Yes,60.0,136.0,64.36,679.2,70.3,4: Every day,4: Every day,1: Yes,0: No,6: 10+ Years,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,22.7,157.0,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incomplete Workbook,0: No,0: No,0: No,0: No,1: Yes,0: No,14.0,14.0
3,9000798,0: No,1: Yes,0: No,0: No,0: No,3: Not limited at all,"2: Yes, limited a little",2: A little bit,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,2: Moderate,3: Daily,4: Always,2: Moderate,2: Moderate,0: None,2: Moderate,0: No,1: Yes,0: No,1: Yes,0: No Limits or avoidance,0: No,0: No,0: No,0: No pain,4: 4,0: No pain,4: 4,30.0,0.0,3: Both IEI and SV,4: Neither,...,0: No,0: No,0: No,0: No,.: Missing Form/Incomplete Workbook,.: Missing Form/Incompl

# Deal with missing values

In [ ]:
## Check amount of missing data in every column

# Create dictionnary
missing_count = {}

# Loop over every column
for i in selected_data.columns:
  if '.: Missing Form/Incomplete Workbook' in selected_data[i].value_counts(): # if column contains missing data
    missing_count[i] = round(selected_data[i].value_counts()['.: Missing Form/Incomplete Workbook'] / selected_data[i].count(), 3) # (number of missing data) / (total data)
  else:
    missing_count[i] = 0

missing_count

{'ID': 0,
 'P01ARTDOC': 0.002,
 'P01ARTDRCV': 0.002,
 'P01ARTHOTH': 0.018,
 'P01BL12SXL': 0.008,
 'P01BL12SXR': 0.012,
 'P01BMI': 0,
 'P01FAMKR': 0.013,
 'P01GOUTCV': 0.02,
 'P01HPL12CV': 0.004,
 'P01HPNL12': 0.003,
 'P01HPNR12': 0.003,
 'P01HPR12CV': 0.003,
 'P01HRSLFXO': 0.998,
 'P01HRSRFXO': 0.997,
 'P01INJR': 0.011,
 'P01KPA30CV': 0.006,
 'P01KPACDCV': 0,
 'P01KPACT30': 0.002,
 'P01KPACTCV': 0.006,
 'P01KPL30CV': 0.004,
 'P01KPMED': 0.001,
 'P01KPMEDCV': 0.002,
 'P01KPNLEV': 0.003,
 'P01KPNREV': 0.005,
 'P01KPR30CV': 0.005,
 'P01KSURGL': 0.002,
 'P01KSURGR': 0.002,
 'P01LKP30CV': 0.002,
 'P01OADEGCV': 0.049,
 'P01OAHIPCV': 0.036,
 'P01OTARTCV': 0.03,
 'P01PMLKRCV': 0.003,
 'P01PMRKRCV': 0.003,
 'P01RAIA': 0.022,
 'P01RAJS1HR': 0.916,
 'P01RAKN6L': 0.917,
 'P01RAKN6R': 0.916,
 'P01RASTASV': 0,
 'P01RAWE6L': 0.916,
 'P01RAWE6R': 0.916,
 'P01RKP30CV': 0.002,
 'P02CNCOTH': 0.993,
 'P02KPMED': 0.001,
 'P02KPMEDCV': 0.004,
 'P02KPNLCV': 0.005,
 'P02KPNRCV': 0.006,
 'P02KSURG': 0.001,
 'P

In [ ]:
## Remove columns with too many missing data
## Arbitrarily set the threshold value at 25%: every column with more than 25% of missing values will be removed
## The data that we have is based on a survey to which patients answered, if less than 75% of the patients answered
## then we consider the features non critical or too difficult to measure.

threshold = 0.25

features_to_remove = []

for i in missing_count:
  if missing_count[i] > threshold:
    features_to_remove.append(i)

## Create new data frame containing only the columns with less than the threshold value of missing data

df = pd.DataFrame()

for i in selected_data.columns:
  if i not in features_to_remove:
    df[i] = selected_data[i]

print('Removed features (too much missing data): ', features_to_remove)
df.head()


Removed features (too much missing data):  ['P01RAJS1HR', 'P01RAKN6R', 'P01RAKN6L', 'V00RAMEDS', 'P02CNCOTH', 'P01RAWE6L', 'P01HRSRFXO', 'P01HRSLFXO', 'P01RAWE6R', 'V00REASW12', 'V00REASW10', 'V00REASW9', 'V00FFQYR86', 'V00RPWKTYP', 'V00LPWKTYP', 'V00NPN400W', 'V00PN400W', 'V00OTH400W']


,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,V00VITMCV,V00RUBCV,V00CAPSNCV,V00BRACCV,V00YOGACV,V00HERBCV,V00RELACV,V00SPIRCV,P01RASTASV,P02KSURGCV,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT
0,9000099,1: Yes,1: Yes,1: Yes,0: No,0: No,3: Not limited at all,"2: Yes, limited a little",1: Not at all,0: None,1: Mild,4: Always,0: Never,1: Mild,1: Mild,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,1: Yes,0: No,1: Yes,1: Yes,2: Limits,0: No,0: No,1: Yes,4: 4,0: No pain,3: 3,2: 2,0.0,14.0,1: SV only,3: Both IEI and SV,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: Does not report RA/inflam arth,0: No,0: No,0: No,0: No,1: Yes,0: No,0: No,1: Yes,0: No,1: Yes,60.0,124.0,276.00,498.9,258.9,0: No vitamins/minerals taken in past year,0: No vitamins/minerals taken in past year,0: No,0: No,23.8,132.0,0: No,0: No,0: No,0: No,0: No,0: No,1: Yes,0: No,22.0,23.0
1,9000296,1: Yes,0: No,0: No,1: Yes,0: No,3: Not limited at all,3: Not limited at all,1: Not at all,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,0: None,0: No,0: No,0: No,0: No,0: No Limits or avoidance,1: Yes,0: No,0: No,0: No pain,0: No pain,0: No pain,0: No pain,0.0,0.0,4: Neither,4: Neither,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: Does not report RA/inflam arth,0: No,0: No,0: No,0: No,1: Yes,0: No,0: No,0: No,0: No,1: Yes,84.0,152.0,73.00,495.5,260.7,1: Didn t take,1: Didn t take,0: No,1: Yes,29.8,190.0,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,12.0,18.0
2,9000622,1: Yes,0: No,1: Yes,1: Yes,0: No,"2: Yes, limited a little","2: Yes, limited a little",3: Moderately,1: Mild,1: Mild,3: Daily,2: Sometimes,0: None,1: Mild,0: None,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,1: Yes,0: No,1: Yes,0: No,2: Limits,1: Yes,0: No,1: Yes,3: 3,0: No pain,2: 2,0: No pain,0.0,20.9,4: Neither,3: Both IEI and SV,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: Does not report RA/inflam arth,0: No,0: No,0: No,0: No,1: Yes,0: No,0: No,0: No,0: No,1: Yes,60.0,136.0,64.36,679.2,70.3,4: Every day,4: Every day,1: Yes,0: No,22.7,157.0,0: No,0: No,0: No,0: No,0: No,0: No,1: Yes,0: No,14.0,14.0
3,9000798,0: No,1: Yes,0: No,0: No,0: No,3: Not limited at all,"2: Yes, limited a little",2: A little bit,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,2: Moderate,2: Moderate,3: Daily,4: Always,2: Moderate,2: Moderate,0: None,2: Moderate,0: No,1: Yes,0: No,1: Yes,0: No Limits or avoidance,0: No,0: No,0: No,0: No pain,4: 4,0: No pain,4: 4,30.0,0.0,3: Both IEI and SV,4: Neither,...,0: No,1: Yes,0: No,0: No,0: No,0: No,0: No,0: No,"1: Report RA/inflam arth, no to all meds",0: No,0: No,0: No,0: No,1: Yes,0: No,0: No,0: No,0: No,1: Yes,86.0,120.0,175.75,834.0,645.9,1: Didn t take,1: Didn t take,0: No,0: No,32.4,250.0,1: Yes,1: Yes,0: No,0: No,0: No,0: No,0: No,1: Yes,22.0,24.0
4,9001104,1: Yes,0: No,1: Yes,1: Yes,0: No,3: Not limited at all,3: Not limited at all,2: A little bit,1: Mild,1: Mild,2: Weekly,1: Rarely,2: Moderate,2: Moderate,1: Mild,0: None,0: None,0: Never,0: Never,0: None,0: None,0: None,3: Severe,0: No,0: No,0: No,0: No,2: Limits,0: No,0: No,1: Yes,5: 5,0: No pain,0: No pain,0: No pain,14.0,33.0,4: Neither,4: Neither,...,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: No,0: Does not report RA/inflam arth,0: No,0: No,0: No,1: Yes,1: Yes,0: No,0: No,0

In [ ]:
## Remove begining of text when it is of the form: 'number: value' --> 'value'

for i in df.columns:
  for j in range(len(df[i])):
    if type(df[i][j]) == str: # if it is string, check if there is a need to remove begining of the content
      content = [char for char in df[i][j]]
      content_isdigit = [char.isdigit() for char in content]

      if content_isdigit[0] == True and content[1] == ':' and content[2] == ' ':
        kept_value = ''.join(content[3:])
        df[i][j] = kept_value

    else: # if the element is not a string, do nothing
      pass

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,V00VITMCV,V00RUBCV,V00CAPSNCV,V00BRACCV,V00YOGACV,V00HERBCV,V00RELACV,V00SPIRCV,P01RASTASV,P02KSURGCV,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT
0,9000099,Yes,Yes,Yes,No,No,Not limited at all,"Yes, limited a little",Not at all,None,Mild,Always,Never,Mild,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,Yes,Limits,No,No,Yes,4,No pain,3,2,0.0,14.0,SV only,Both IEI and SV,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,Yes,No,Yes,60.0,124.0,276.00,498.9,258.9,No vitamins/minerals taken in past year,No vitamins/minerals taken in past year,No,No,23.8,132.0,No,No,No,No,No,No,Yes,No,22.0,23.0
1,9000296,Yes,No,No,Yes,No,Not limited at all,Not limited at all,Not at all,None,None,Never,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,Yes,No,No,No pain,No pain,No pain,No pain,0.0,0.0,Neither,Neither,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,84.0,152.0,73.00,495.5,260.7,Didn t take,Didn t take,No,Yes,29.8,190.0,No,No,No,No,No,No,No,No,12.0,18.0
2,9000622,Yes,No,Yes,Yes,No,"Yes, limited a little","Yes, limited a little",Moderately,Mild,Mild,Daily,Sometimes,None,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,No,Limits,Yes,No,Yes,3,No pain,2,No pain,0.0,20.9,Neither,Both IEI and SV,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,60.0,136.0,64.36,679.2,70.3,Every day,Every day,Yes,No,22.7,157.0,No,No,No,No,No,No,Yes,No,14.0,14.0
3,9000798,No,Yes,No,No,No,Not limited at all,"Yes, limited a little",A little bit,None,None,Never,Never,None,None,None,Moderate,Moderate,Daily,Always,Moderate,Moderate,None,Moderate,No,Yes,No,Yes,No Limits or avoidance,No,No,No,No pain,4,No pain,4,30.0,0.0,Both IEI and SV,Neither,...,No,Yes,No,No,No,No,No,No,"Report RA/inflam arth, no to all meds",No,No,No,No,Yes,No,No,No,No,Yes,86.0,120.0,175.75,834.0,645.9,Didn t take,Didn t take,No,No,32.4,250.0,Yes,Yes,No,No,No,No,No,Yes,22.0,24.0
4,9001104,Yes,No,Yes,Yes,No,Not limited at all,Not limited at all,A little bit,Mild,Mild,Weekly,Rarely,Moderate,Moderate,Mild,None,None,Never,Never,None,None,None,Severe,No,No,No,No,Limits,No,No,Yes,5,No pain,No pain,No pain,14.0,33.0,Neither,Neither,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,Yes,Yes,No,No,No,No,Yes,84.0,112.0,256.40,214.8,82.5,A few days per month,A few days per month,Yes,No,30.7,164.0,Yes,No,No,No,No,No,No,No,15.0,13.0


In [ ]:
## For each column, replace missing data by most common observation in the column

for col in df.columns:
    
  if df[col].dtype == 'int64' or df[col].dtype == 'float64':
    most_common_observation = df[col].describe()['mean']
  else:
    most_common_observation = df[col].describe()['top']
  
  for index in range(len(df[col])):
    if df[col][index] == '.: Missing Form/Incomplete Workbook':
      df[col][index] = most_common_observation

df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/i

,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,V00VITMCV,V00RUBCV,V00CAPSNCV,V00BRACCV,V00YOGACV,V00HERBCV,V00RELACV,V00SPIRCV,P01RASTASV,P02KSURGCV,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT
0,9000099,Yes,Yes,Yes,No,No,Not limited at all,"Yes, limited a little",Not at all,None,Mild,Always,Never,Mild,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,Yes,Limits,No,No,Yes,4,No pain,3,2,0.0,14.0,SV only,Both IEI and SV,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,Yes,No,Yes,60.0,124.0,276.00,498.9,258.9,No vitamins/minerals taken in past year,No vitamins/minerals taken in past year,No,No,23.8,132.0,No,No,No,No,No,No,Yes,No,22.0,23.0
1,9000296,Yes,No,No,Yes,No,Not limited at all,Not limited at all,Not at all,None,None,Never,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,Yes,No,No,No pain,No pain,No pain,No pain,0.0,0.0,Neither,Neither,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,84.0,152.0,73.00,495.5,260.7,Didn t take,Didn t take,No,Yes,29.8,190.0,No,No,No,No,No,No,No,No,12.0,18.0
2,9000622,Yes,No,Yes,Yes,No,"Yes, limited a little","Yes, limited a little",Moderately,Mild,Mild,Daily,Sometimes,None,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,No,Limits,Yes,No,Yes,3,No pain,2,No pain,0.0,20.9,Neither,Both IEI and SV,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,60.0,136.0,64.36,679.2,70.3,Every day,Every day,Yes,No,22.7,157.0,No,No,No,No,No,No,Yes,No,14.0,14.0
3,9000798,No,Yes,No,No,No,Not limited at all,"Yes, limited a little",A little bit,None,None,Never,Never,None,None,None,Moderate,Moderate,Daily,Always,Moderate,Moderate,None,Moderate,No,Yes,No,Yes,No Limits or avoidance,No,No,No,No pain,4,No pain,4,30.0,0.0,Both IEI and SV,Neither,...,No,Yes,No,No,No,No,No,No,"Report RA/inflam arth, no to all meds",No,No,No,No,Yes,No,No,No,No,Yes,86.0,120.0,175.75,834.0,645.9,Didn t take,Didn t take,No,No,32.4,250.0,Yes,Yes,No,No,No,No,No,Yes,22.0,24.0
4,9001104,Yes,No,Yes,Yes,No,Not limited at all,Not limited at all,A little bit,Mild,Mild,Weekly,Rarely,Moderate,Moderate,Mild,None,None,Never,Never,None,None,None,Severe,No,No,No,No,Limits,No,No,Yes,5,No pain,No pain,No pain,14.0,33.0,Neither,Neither,...,No,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,Yes,Yes,No,No,No,No,Yes,84.0,112.0,256.40,214.8,82.5,A few days per month,A few days per month,Yes,No,30.7,164.0,Yes,No,No,No,No,No,No,No,15.0,13.0


# Building of the outcome

In [ ]:
#We upload the data sets with the information every year

raw_data_1years = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical01.txt', sep='|')
raw_data_2years = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical03.txt', sep='|')
raw_data_3years = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical05.txt', sep='|')
raw_data_4years = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical06.txt', sep='|')
raw_data_5years = pd.read_csv('/content/drive/MyDrive/Osteoarthritis dataset/All Clinical/AllClinical07.txt', sep='|')

#The outcome of the initial data set is the feature P01OADEGCV
#The outcome that we consider for each data set is the features V01ARTH12, V03ARTH12, V05ARTH12, V06ARTH12, V07ARTH12 respectively

In [ ]:
#Creation of a data frame with only the outcomes of the 5 years
#The aim is to add a column where the value is equal to one if the patient has been declared as having arthritis in the past 5 years
#Hence, if for a patient, the value of the outcome is 1 in one of the 5 outcomes we have, the resulting value in 1

outcome_1years = raw_data_1years['V01ARTH12']
outcome_2years = raw_data_2years['V03ARTH12']
outcome_3years = raw_data_3years['V05ARTH12']
outcome_4years = raw_data_4years['V06ARTH12']
outcome_5years = raw_data_5years['V07ARTH12']

outcomes = pd.concat([outcome_1years, outcome_2years, outcome_3years, outcome_4years, outcome_5years], axis = 1, ignore_index=True)
outcomes.columns = ['Year1', 'Year2', 'Year3', 'Year4', 'Year5']
outcomes.head(10)

,Year1,Year2,Year3,Year4,Year5
0,1: Yes,0.0,0.0,0.0,0.0
1,0: No,0.0,0.0,0.0,0.0
2,1: Yes,NaN,NaN,NaN,NaN
3,0: No,0.0,0.0,0.0,0.0
4,0: No,1.0,NaN,NaN,NaN
5,0: No,0.0,0.0,0.0,0.0
6,1: Yes,1.0,NaN,1.0,0.0
7,0: No,0.0,1.0,0.0,0.0
8,1: Yes,0.0,0.0,0.0,0.0
9,0: No,0.0,1.0,1.0,1.0


In [ ]:
#Clean the first column of the data set ouctomes: we will only keep 0 or 1
for j in range(len(outcomes['Year1'])):
  content = [char for char in outcomes['Year1'][j]]

  if content[0] == '0':
    outcomes['Year1'][j] = 0.0

  if content[0] == '1':
    outcomes['Year1'][j] = 1.0
  
outcomes.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Year1,Year2,Year3,Year4,Year5
0,1,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0
2,1,NaN,NaN,NaN,NaN
3,0,0.0,0.0,0.0,0.0
4,0,1.0,NaN,NaN,NaN
5,0,0.0,0.0,0.0,0.0
6,1,1.0,NaN,1.0,0.0
7,0,0.0,1.0,0.0,0.0
8,1,0.0,0.0,0.0,0.0
9,0,0.0,1.0,1.0,1.0


In [ ]:
#Add a new column which will be equal to 1 if one of the ouctome over the past 5 years is equal to one
cumulative_outcome = pd.DataFrame(index = range(len(outcomes)), columns = ['cumulative_outcome'])
for j in range(len(outcomes)):
  if type(outcomes['Year1'][j]) == str and type(outcomes['Year2'][j]) == str and type(outcomes['Year3'][j]) == str and type(outcomes['Year4'][j]) == str and type(outcomes['Year5'][j]) == str:
    cumulative_outcome['cumulative_outcome'][j] = 'Missing Form/Incomplete Workbook'
  if outcomes['Year1'][j] == 1 or outcomes['Year2'][j] == 1.0 or outcomes['Year3'][j] == 1.0 or outcomes['Year4'][j] == 1.0 or outcomes['Year5'][j] == 1.0:
    cumulative_outcome['cumulative_outcome'][j] = 'Yes'
    pass
  else:
    cumulative_outcome['cumulative_outcome'][j] = 'No'

df2 = pd.concat([df, cumulative_outcome], axis = 1)
df2.head(20)

,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,V00RUBCV,V00CAPSNCV,V00BRACCV,V00YOGACV,V00HERBCV,V00RELACV,V00SPIRCV,P01RASTASV,P02KSURGCV,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT,cumulative_outcome
0,9000099,Yes,Yes,Yes,No,No,Not limited at all,"Yes, limited a little",Not at all,None,Mild,Always,Never,Mild,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,Yes,Limits,No,No,Yes,4,No pain,3,2,0.0,14.0,SV only,Both IEI and SV,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,Yes,No,Yes,60.0,124.0,276.00,498.9,258.9,No vitamins/minerals taken in past year,No vitamins/minerals taken in past year,No,No,23.8,132.0,No,No,No,No,No,No,Yes,No,22.0,23.0,Yes
1,9000296,Yes,No,No,Yes,No,Not limited at all,Not limited at all,Not at all,None,None,Never,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,Yes,No,No,No pain,No pain,No pain,No pain,0.0,0.0,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,84.0,152.0,73.00,495.5,260.7,Didn t take,Didn t take,No,Yes,29.8,190.0,No,No,No,No,No,No,No,No,12.0,18.0,No
2,9000622,Yes,No,Yes,Yes,No,"Yes, limited a little","Yes, limited a little",Moderately,Mild,Mild,Daily,Sometimes,None,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,No,Limits,Yes,No,Yes,3,No pain,2,No pain,0.0,20.9,Neither,Both IEI and SV,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,60.0,136.0,64.36,679.2,70.3,Every day,Every day,Yes,No,22.7,157.0,No,No,No,No,No,No,Yes,No,14.0,14.0,Yes
3,9000798,No,Yes,No,No,No,Not limited at all,"Yes, limited a little",A little bit,None,None,Never,Never,None,None,None,Moderate,Moderate,Daily,Always,Moderate,Moderate,None,Moderate,No,Yes,No,Yes,No Limits or avoidance,No,No,No,No pain,4,No pain,4,30.0,0.0,Both IEI and SV,Neither,...,Yes,No,No,No,No,No,No,"Report RA/inflam arth, no to all meds",No,No,No,No,Yes,No,No,No,No,Yes,86.0,120.0,175.75,834.0,645.9,Didn t take,Didn t take,No,No,32.4,250.0,Yes,Yes,No,No,No,No,No,Yes,22.0,24.0,No
4,9001104,Yes,No,Yes,Yes,No,Not limited at all,Not limited at all,A little bit,Mild,Mild,Weekly,Rarely,Moderate,Moderate,Mild,None,None,Never,Never,None,None,None,Severe,No,No,No,No,Limits,No,No,Yes,5,No pain,No pain,No pain,14.0,33.0,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,Yes,Yes,No,No,No,No,Yes,84.0,112.0,256.40,214.8,82.5,A few days per month,A few days per month,Yes,No,30.7,164.0,Yes,No,No,No,No,No,No,No,15.0,13.0,Yes
5,9001400,No,No,No,Yes,No,Not limited at all,Not limited at all,Not at all,None,None,Weekly,Never,None,Mild,Mild,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,No,No,No,3,No pain,2,No pain,0.0,3.0,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,62.0,118.0,158.62,961.6,141.5,Every day,Every day,Yes,No,23.5,128.0,No,No,No,No,No,No,No,No,8.0,7.0,No
6,9001695,Yes,No,No,No,No,Not limited at all,Not limited at all,Not at all,None,None,Monthly,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,No,No,No,1,No pain,1,No pain,0.0,NaN,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,70.0,115.0,13.01,447.2,63.1,No vitamins/minerals taken in past y

In [ ]:
#Drop the rows with P01OADEGCV = Yes, because it means the patient already has arthritis at the beginning
#Drop the rows where we have missing forms (there are some missing forms for the cumulative_outcome feature)
index_yes = df2[df2['P01OADEGCV'] == 'Yes'].index
print(len(index_yes))
index_missing = df2[df2['cumulative_outcome'] == 'Missing Form/Incomplete Workbook'].index
print(len(index_missing))
df2.drop(index_yes, inplace = True)
df2.drop(index_missing, inplace = True)
df2.head()

1094
0


,ID,P01KPNREV,P01KPNLEV,P01KPACT30,P01HPNR12,P01HPNL12,V00SF2,V00SF3,V00SF8,V00WPRKN1,V00WPRKN2,V00P7RKFR,V00KSXRKN1,V00DIRKN1,V00DIRKN2,V00DIRKN14,V00WPLKN1,V00WPLKN2,V00P7LKFR,V00KSXLKN1,V00DILKN1,V00DILKN2,V00DILKN14,V00KOOSFX5,P02KPNRCV,P02KPNLCV,P01KPR30CV,P01KPL30CV,P01KPACTCV,P01HPR12CV,P01HPL12CV,P01KPA30CV,V00P7RKRCV,V00P7LKRCV,P01PMRKRCV,P01PMLKRCV,V00WOMTSL,V00WOMTSR,P01BL12SXL,P01BL12SXR,...,V00RUBCV,V00CAPSNCV,V00BRACCV,V00YOGACV,V00HERBCV,V00RELACV,V00SPIRCV,P01RASTASV,P02KSURGCV,V00OTHCAMC,V00OTHCAM,V00DISCOMF,V00LFXCOMP,V00LFXPN,V00EKRSL,V00REXPN,V00EKRSR,V00RFXCOMP,V00BPDIAS,V00BPSYS,V00DTVITD,V00DTCALC,V00DTCHOL,V00VITDCV,V00CALCMCV,V00VIT9,V00VIT1,P01BMI,V00PASE,P01FAMKR,P02WTGA,V00RKEFFB,V00RKEFFPT,V00LKEFFB,V00LKEFFPT,V00RKRFXPN,V00LKRFXPN,V00LLWGT,V00RLWGT,cumulative_outcome
1,9000296,Yes,No,No,Yes,No,Not limited at all,Not limited at all,Not at all,None,None,Never,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,Yes,No,No,No pain,No pain,No pain,No pain,0.0,0.0,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,84.0,152.0,73.00,495.5,260.7,Didn t take,Didn t take,No,Yes,29.8,190.0,No,No,No,No,No,No,No,No,12.0,18.0,No
2,9000622,Yes,No,Yes,Yes,No,"Yes, limited a little","Yes, limited a little",Moderately,Mild,Mild,Daily,Sometimes,None,Mild,None,None,None,Never,Never,None,None,None,Moderate,Yes,No,Yes,No,Limits,Yes,No,Yes,3,No pain,2,No pain,0.0,20.9,Neither,Both IEI and SV,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,60.0,136.0,64.36,679.2,70.3,Every day,Every day,Yes,No,22.7,157.0,No,No,No,No,No,No,Yes,No,14.0,14.0,Yes
6,9001695,Yes,No,No,No,No,Not limited at all,Not limited at all,Not at all,None,None,Monthly,Never,None,None,None,None,None,Never,Never,None,None,None,None,No,No,No,No,No Limits or avoidance,No,No,No,1,No pain,1,No pain,0.0,NaN,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,70.0,115.0,13.01,447.2,63.1,No vitamins/minerals taken in past year,No vitamins/minerals taken in past year,No,No,28.6,280.0,Yes,No,Yes,Yes,No,No,No,No,13.0,13.0,Yes
7,9001897,No,No,No,No,No,Not limited at all,Not limited at all,Not at all,None,None,Never,Never,None,None,None,None,None,Never,Never,None,None,None,None,Yes,No,No,No,No Limits or avoidance,No,No,No,No pain,No pain,No pain,No pain,0.0,0.0,Neither,IEI only,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,82.0,150.0,228.89,785.9,57.7,1-3 days per week,1-3 days per week,Yes,Yes,25.9,156.0,No,No,Yes,No,No,No,No,No,16.0,17.0,Yes
10,9002411,Yes,No,No,No,No,"Yes, limited a little","Yes, limited a little",A little bit,None,Mild,Never,Never,None,Mild,None,None,None,Never,Never,None,Mild,None,None,No,No,No,No,No Limits or avoidance,No,No,No,No pain,No pain,No pain,No pain,1.1,2.1,Neither,Neither,...,No,No,No,No,No,No,No,Does not report RA/inflam arth,No,No,No,No,Yes,No,No,No,No,Yes,60.0,110.0,29.25,551.3,164.4,Didn t take,Didn t take,No,No,31.8,54.0,No,Yes,No,No,No,No,No,No,18.0,22.0,No


In [ ]:
df2.to_csv('cleaned_data.csv', index=False)